In [1]:
import pandas as pd

amanda_psms = pd.read_excel("msamanda_percolator_1%.xlsx")

In [2]:
import re

mod_dict = {"Carbamidomethyl": "57.021464", "Oxidation": "15.994915"}

def get_peptidoform(row):
    seq = str(row["Annotated Sequence"]).upper()
    if str(row["Modifications"]) == "nan":
        return seq + "[]"
    seq += "["
    for modification in str(row["Modifications"]).split(";"):
        mod = modification.strip()
        if mod != "":
            modmass = mod_dict[mod.split("(")[1].split(")")[0].strip()]
            modplace = int(re.sub(r"[^0-9]", "", mod.split("(")[0]).strip()) - 1
            seq += str(modplace) + ":" + modmass + "+"
    seq = seq.rstrip("+")
    seq += "]"
    return seq

In [3]:
psms = dict()

for i, row in amanda_psms.iterrows():
    psms[int(row["First Scan"])] = get_peptidoform(row)

In [4]:
def get_candidates(filename, top_n, nr_lines = "all", start = 0):
    candidates = dict()

    with open(filename, "r", encoding = "utf-8") as f:
        lines = f.readlines()
        f.close()

    if nr_lines == "all":
        for line in lines[1:]:
            values = line.split(";")
            candidates[int(values[0])] = [val.replace("_", "") for val in values[1].split(",")[:top_n]]
    else:
        for i, line in enumerate(lines[1:]):
            if i > start:
                if i >= nr_lines:
                    break
                values = line.split(";")
                candidates[int(values[0])] = [val.replace("_", "") for val in values[1].split(",")[:top_n]]

    return candidates

In [5]:
def get_coverage(psms, candidates, top_n):
    found_candidates = 0
    
    for psm in psms:
        candidates_for_scan = candidates[psm][:top_n]
        pep = psms[psm]
        if pep in candidates_for_scan:
            found_candidates += 1
    
    print(found_candidates/len(psms))
    return

In [6]:
def get_coverage_lines(psms, candidates, top_n):
    found_candidates = 0
    total_psms_searched = 0
    
    for psm in psms:
        if psm in candidates:
            total_psms_searched += 1
            candidates_for_scan = candidates[psm][:top_n]
            pep = psms[psm]
            if pep in candidates_for_scan:
                found_candidates += 1
    
    print(found_candidates/total_psms_searched)
    return

In [7]:
candidates = get_candidates("test_pepf.csv", 1000)
get_coverage(psms, candidates, 1000)
get_coverage(psms, candidates, 500)
get_coverage(psms, candidates, 100)
get_coverage(psms, candidates, 50)

0.9709368751785205
0.9597972007997715
0.925235646958012
0.9036703798914596
